### Dataset

In [1]:
import pandas as pd
from auto_ts import auto_timeseries
import dill
import talib
import numpy as np
import warnings

warnings.filterwarnings('ignore')

from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam, RMSprop, SGD
from scikeras.wrappers import KerasRegressor
from keras.callbacks import EarlyStopping
from skopt import BayesSearchCV
from sklearn.preprocessing import LabelEncoder
import tensorflow.keras.backend as K
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from keras.losses import categorical_crossentropy
from sklearn.metrics import log_loss
from keras.utils import to_categorical

Imported auto_timeseries version:0.0.90. Call by using:
model = auto_timeseries(score_type='rmse',
        time_interval='M', non_seasonal_pdq=None, seasonality=False,
        seasonal_period=12, model_type=['best'], verbose=2, dask_xgboost_flag=0)
model.fit(traindata, ts_column,target)
model.predict(testdata, model='best')



2024-05-13 09:04:09.148880: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Evito que ciertas columnas se transformen a notacion cientifica en las predicciones
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

In [3]:
columns = [
    'Open_time',
    'Open',
    'High',
    'Low',
    # 'Close',
    'Number of trades',
    'Close_BTCUSDT',
    'Volume_BTCUSDT',
    'Number_of_trades_BTCUSDT',
    'Close_ETHUSDT',
    'Volume_ETHUSDT',
    'Number_of_trades_ETHUSDT',
    'Close_BNBUSDT',
    'Volume_BNBUSDT',
    'Number_of_trades_BNBUSDT',
    'SMA_20',
    'EMA_20',
    'Upper_Band',
    'Middle_Band',
    'Lower_Band',
    'RSI',
    'buy_1000x_high_coinbase',
    'sell_1000x_high_coinbase',
    'total_trades_coinbase',	
    'Tweets_Utilizados',
    'Tweets_Utilizados_coin',
    'Tweets_Utilizados_referentes',
    'Tweets_Utilizados_whale_alert',
    'Buy_1000x_high',
    'sell_1000x_high',
    'total_trades_binance'
]

### Armado y entrenamiento de un clasificador a partir de los datos originales

#### Modelo LSTM

In [4]:
complete_dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv') 
classifier_dataset = complete_dataset[columns]
classifier_dataset['Open_time'] = pd.to_datetime(classifier_dataset['Open_time'])
classifier_dataset['Tendencia'] = complete_dataset['Tendencia']

clasifier_validation = classifier_dataset[-10:]
classifier_dataset = classifier_dataset[:-10]

In [5]:
display(classifier_dataset.tail())

,Open_time,Open,High,Low,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Tendencia
941,2024-04-25,6.93,7.00,6.70,71088.00,64498.34,31341.46,1375324.00,3155.80,352288.55,861077.00,613.20,453745.52,353114.00,7.43,7.45,9.08,7.43,5.77,38.83,21.00,26.00,33468.00,151,114,0.00,22.00,242.00,219.00,48000.00,Lateral
942,2024-04-26,6.86,6.95,6.71,67383.00,63770.01,27085.19,1025561.00,3131.30,252522.65,628635.00,598.00,302119.88,269508.00,7.34,7.38,8.94,7.34,5.74,37.81,29.00,24.00,26619.00,117,106,0.00,14.00,292.00,324.00,42000.00,Lateral
943,2024-04-27,6.76,6.87,6.51,64779.00,63461.98,20933.06,912422.00,3255.56,323811.19,734026.00,596.20,268783.91,233820.00,7.24,7.33,8.73,7.24,5.76,38.57,17.00,17.00,25565.00,101,138,0.00,7.00,248.00,179.00,41000.00,Lateral
944,2024-04-28,6.81,6.95,6.69,43208.00,63118.62,16949.20,790652.00,3263.45,304766.01,753239.00,600.20,258059.43,206703.00,7.13,7.27,8.38,7.13,5.88,37.66,16.00,20.00,20954.00,82,106,0.00,13.00,173.00,165.00,26000.00,Lateral
945,2024-04-29,6.73,6.83,6.47,63006.00,63866.00,28150.23,1152296.00,3216.73,421831.29,943719.00,592.80,330474.01,271926.00,7.03,7.20,8.08,7.03,5.97,36.02,69.00,37.00,33959.00,115,125,0.00,24.00,260.00,188.00,41000.00,Bajista


In [6]:
classifier_dataset.shape

(946, 31)

In [7]:
X = classifier_dataset.drop(columns=["Tendencia", "Open_time"])
y = classifier_dataset["Tendencia"]

# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(classifier_dataset["Tendencia"])

y = y.to_numpy().reshape(-1, 1)
onehot_encoder = OneHotEncoder(sparse=False)
y_one_hot = onehot_encoder.fit_transform(y)


In [8]:
display(y_one_hot)

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [9]:
import warnings
warnings.filterwarnings('ignore')

def create_model(activation, units, dropout, learning_rate, l2_penalty, depth, optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units=int(units/2), activation=activation, input_shape=(len(X.columns), 1), return_sequences=True, kernel_regularizer=l2(l2_penalty)))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    
    for _ in range(depth - 1):
        model.add(LSTM(units=units, activation=activation, return_sequences=True, kernel_regularizer=l2(l2_penalty)),)
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
    
    model.add(LSTM(units=int(units*2), activation=activation, kernel_regularizer=l2(l2_penalty)))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(units=3, activation='softmax'))
    
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
regressor = KerasRegressor(build_fn=create_model, verbose=0, activation='relu', units=50, dropout=0.2, learning_rate=0.1, l2_penalty=0.001, depth=2, optimizer='adam')

cv = TimeSeriesSplit(n_splits=10).split(X)
param_space = {
    'depth': [2, 3, 4, 5],
    'activation': ['relu', 'tanh', 'swish', 'selu'],
    'units': [64, 128, 256, 512],
    'dropout': [0.1, 0.2, 0.3, 0.4],
    'learning_rate': [0.01, 0.001, 0.0001],
    'epochs': [10, 20, 30, 50, 100],
    'batch_size': [32, 64, 128],
    'optimizer': ['adam', 'rmsprop', 'sgd'],
    'l2_penalty': [0.001, 0.01, 0.1]
}

def categorical_crossentropy_loss(estimator, X_test, y_test):
    y_pred = estimator.predict(X_test)
    
    if np.isnan(y_pred).any():
        y_pred[np.isnan(y_pred)] = 0
    
    loss = log_loss(y_test, y_pred)
    return loss

bayes_search = BayesSearchCV(regressor, param_space, scoring=categorical_crossentropy_loss, cv=cv, verbose=0)#10)
bayes_result = bayes_search.fit(X, y_one_hot, callbacks=[early_stopping])

In [28]:
# Show best results
print("Best score:", bayes_result.best_score_)
print("Best parameters:", bayes_result.best_params_)

# Entrenar el modelo con los mejores hiperparámetros
best_model = bayes_result.best_estimator_
best_model.fit(X, y_one_hot)

Best score: 5.120278119888486
Best parameters: OrderedDict([('activation', 'selu'), ('batch_size', 64), ('depth', 4), ('dropout', 0.1), ('epochs', 10), ('l2_penalty', 0.001), ('learning_rate', 0.001), ('optimizer', 'rmsprop'), ('units', 512)])


KerasRegressor(
	model=None
	build_fn=<function create_model at 0x7f8ba8b54040>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=64
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	activation=selu
	units=512
	dropout=0.1
	learning_rate=0.001
	l2_penalty=0.001
	depth=4
)

In [29]:
import json

# Obtener los hiperparámetros y puntajes de los 5 mejores modelos
top_n_models = 5
best_params_list = []
best_scores_list = []

for i in range(min(top_n_models, len(bayes_search.cv_results_['params']))):
    best_params_list.append(bayes_search.cv_results_['params'][i])
    best_scores_list.append(bayes_search.cv_results_['mean_test_score'][i])

# Guardar los hiperparámetros de los 5 mejores modelos en un archivo JSON
with open('lstm_classifier/top_5_hyperparameters.json', 'w') as f:
    json.dump({'best_params': best_params_list, 'best_scores': best_scores_list}, f)

# O imprimir los hiperparámetros
print("Top 5 mejores modelos:")
for i in range(len(best_params_list)):
    print("Modelo", i+1)
    print("Hiperparámetros:", best_params_list[i])
    print("Puntaje:", best_scores_list[i])


Top 5 mejores modelos:
Modelo 1
Hiperparámetros: OrderedDict([('activation', 'relu'), ('batch_size', 64), ('depth', 4), ('dropout', 0.2), ('epochs', 30), ('l2_penalty', 0.01), ('learning_rate', 0.0001), ('optimizer', 'adam'), ('units', 64)])
Puntaje: 1.157736949168703
Modelo 2
Hiperparámetros: OrderedDict([('activation', 'selu'), ('batch_size', 32), ('depth', 4), ('dropout', 0.4), ('epochs', 20), ('l2_penalty', 0.1), ('learning_rate', 0.01), ('optimizer', 'sgd'), ('units', 128)])
Puntaje: 1.1129858996812934
Modelo 3
Hiperparámetros: OrderedDict([('activation', 'relu'), ('batch_size', 64), ('depth', 5), ('dropout', 0.4), ('epochs', 20), ('l2_penalty', 0.1), ('learning_rate', 0.0001), ('optimizer', 'sgd'), ('units', 64)])
Puntaje: 1.1185720457866446
Modelo 4
Hiperparámetros: OrderedDict([('activation', 'relu'), ('batch_size', 128), ('depth', 3), ('dropout', 0.3), ('epochs', 50), ('l2_penalty', 0.1), ('learning_rate', 0.001), ('optimizer', 'sgd'), ('units', 512)])
Puntaje: 2.2327541293501

#### Armado del ensamble

In [10]:
def generate_prime_seeds(n):
    seeds = []
    num = 70001  # Comenzamos desde el primer número primo mayor que 70000
    while len(seeds) < n:
        is_prime = True
        for i in range(2, int(num**0.5) + 1):
            if num % i == 0:
                is_prime = False
                break
        if is_prime:
            seeds.append(num)
        num += 1
    return seeds


In [11]:
## Clase personalizada para hacer el ensamble, dado que sklearn no provee ninguna clase que permita hacer ensmble
## de modelos re regresion multivariados
class MultivariableVotingRegressor:
    def __init__(self, models):
        self.models = models

    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)

    def predict(self, X):
        # Hacer predicciones con cada modelo
        predictions = np.array([model.predict(X) for model in self.models])
        
        # Calcular la moda de las predicciones
        mode_predictions = np.argmax(np.sum(predictions, axis=0), axis=1)
        
        return mode_predictions


In [12]:

import json

# Leer los hiperparámetros desde el archivo JSON
with open('lstm_classifier/top_5_hyperparameters.json', 'r') as f:
    top_hyperparameters = json.load(f)

prime_seeds = generate_prime_seeds(30)
models = []
best_seeds= {}

# Train models with different seeds for each set of hyperparameters
for mode_number, params in enumerate(top_hyperparameters['best_params']):
    best_validation_errors = {}
    
    for seed_number, seed in enumerate(prime_seeds):
        model = KerasRegressor(build_fn=create_model, random_state=seed, verbose=1, **params)
        model.fit(X, y_one_hot)
        
        # Make predictions with the model
        model_predictions = model.predict(X)
        
        # Calculate error (training error)
        train_error = categorical_crossentropy(y_one_hot, model_predictions)
        
        mean_train_error = np.mean(train_error)

        # Update best validation error for this seed
        best_validation_errors[seed] = mean_train_error
    
        print(f"model number {mode_number}, seed number {seed_number}")
    # print("Best validation errors:", best_validation_errors)

    # Find the best seed for this set of hyperparameters
    best_seed_for_params = min(best_validation_errors, key=lambda k: best_validation_errors[k])
    best_seeds[str(params)] = best_seed_for_params
    
    # Create and train the model with the best seed
    model = KerasRegressor(build_fn=create_model, random_state=best_seed_for_params, verbose=1, **params)
    model.fit(X, y_one_hot)
    models.append(model)

# Define and train the ensemble model
ensemble = MultivariableVotingRegressor(models)
ensemble.fit(X, y_one_hot)

# Save the best seeds to a JSON file
with open('lstm_classifier/best_seeds.json', 'w') as f:
    json.dump(best_seeds, f)    

#### Clasificacion con el ensamble sobre las redicciones de los modelos generativos

In [13]:
with open('lstm_classifier/best_seeds.json', 'r') as f:
    best_seeds = json.load(f)

# 21 Crear y entrenar los modelos con los hiperparámetros y semillas guardados
models = []
for params_str, seed in best_seeds.items():
    params = json.loads(params_str.replace("'", "\""))
    model = KerasRegressor(build_fn=create_model, random_state=seed, **params)
    model.fit(X, y_one_hot)
    models.append(model)

Epoch 1/30
15/15 [==============================] - 14s 113ms/step - loss: 5.3906 - accuracy: 0.3140
Epoch 2/30
15/15 [==============================] - 2s 113ms/step - loss: 5.2490 - accuracy: 0.3467
Epoch 3/30
15/15 [==============================] - 2s 127ms/step - loss: 5.1251 - accuracy: 0.3879
Epoch 4/30
15/15 [==============================] - 2s 133ms/step - loss: 5.1418 - accuracy: 0.3869
Epoch 5/30
15/15 [==============================] - 2s 147ms/step - loss: 5.0897 - accuracy: 0.3584
Epoch 6/30
15/15 [==============================] - 2s 145ms/step - loss: 5.0094 - accuracy: 0.4059
Epoch 7/30
15/15 [==============================] - 3s 180ms/step - loss: 5.0322 - accuracy: 0.3805
Epoch 8/30
15/15 [==============================] - 3s 181ms/step - loss: 4.9316 - accuracy: 0.3953
Epoch 9/30
15/15 [==============================] - 3s 208ms/step - loss: 4.9339 - accuracy: 0.3753
Epoch 10/30
15/15 [==============================] - 3s 209ms/step - loss: 4.8933 - accuracy: 0.414

In [14]:
ensemble = MultivariableVotingRegressor(models)
ensemble.fit(X, y_one_hot)

Epoch 1/30
15/15 [==============================] - 13s 199ms/step - loss: 5.3906 - accuracy: 0.3140
Epoch 2/30
15/15 [==============================] - 3s 185ms/step - loss: 5.2490 - accuracy: 0.3467
Epoch 3/30
15/15 [==============================] - 3s 199ms/step - loss: 5.1251 - accuracy: 0.3879
Epoch 4/30
15/15 [==============================] - 2s 162ms/step - loss: 5.1418 - accuracy: 0.3869
Epoch 5/30
15/15 [==============================] - 3s 186ms/step - loss: 5.0897 - accuracy: 0.3584
Epoch 6/30
15/15 [==============================] - 3s 180ms/step - loss: 5.0094 - accuracy: 0.4059
Epoch 7/30
15/15 [==============================] - 3s 169ms/step - loss: 5.0322 - accuracy: 0.3805
Epoch 8/30
15/15 [==============================] - 3s 223ms/step - loss: 4.9316 - accuracy: 0.3953
Epoch 9/30
15/15 [==============================] - 4s 238ms/step - loss: 4.9339 - accuracy: 0.3753
Epoch 10/30
15/15 [==============================] - 3s 216ms/step - loss: 4.8933 - accuracy: 0.414

##### Datos originales:

In [16]:
display(clasifier_validation)

,Open_time,Open,High,Low,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Tendencia
946,2024-04-30,6.59,6.67,6.04,115512.00,60672.00,54947.66,1985671.00,3014.05,561717.49,1292873.00,578.40,766513.45,486465.00,6.93,7.13,7.80,6.93,6.05,34.18,51.00,55.00,48709.00,142,187,1.00,23.00,379.00,377.00,70000.00,Bajista
947,2024-05-01,6.42,6.93,6.13,175570.00,58364.97,81166.47,2401089.00,2972.46,624963.78,1365039.00,561.80,669027.32,427425.00,6.85,7.11,7.41,6.85,6.29,43.30,42.00,50.00,83718.00,130,177,0.00,36.00,327.00,340.00,107000.00,Alcista
948,2024-05-02,6.90,7.41,6.69,109002.00,59060.61,47583.82,1572898.00,2986.19,365939.72,880167.00,560.50,359794.32,250921.00,6.85,7.12,7.42,6.85,6.29,49.27,87.00,57.00,61208.00,461,374,1.00,25.00,270.00,282.00,71000.00,Alcista
949,2024-05-03,7.27,7.39,7.00,109634.00,62882.01,43628.40,1558661.00,3102.61,355825.84,859542.00,587.00,342906.43,257575.00,6.90,7.14,7.44,6.90,6.35,48.86,52.00,40.00,46255.00,573,474,1.00,22.00,386.00,635.00,69000.00,Lateral
950,2024-05-04,7.24,7.28,7.11,71120.00,63892.04,24368.69,1113509.00,3117.23,196263.95,575026.00,585.70,197129.25,210303.00,6.91,7.13,7.46,6.91,6.36,46.98,68.00,50.00,34251.00,407,472,0.00,14.00,203.00,232.00,49000.00,Bajista
951,2024-05-05,7.12,7.40,7.01,72928.00,64012.00,18526.75,992921.00,3136.41,218760.27,600693.00,592.00,180458.24,180794.00,6.94,7.15,7.51,6.94,6.37,50.00,37.00,52.00,29197.00,417,499,0.00,6.00,320.00,284.00,47000.00,Alcista
952,2024-05-06,7.30,7.47,7.06,94264.00,63165.19,34674.92,1392557.00,3062.60,355135.30,873200.00,588.20,278669.01,248490.00,6.96,7.15,7.53,6.96,6.39,47.10,49.00,71.00,40027.00,482,531,0.00,25.00,339.00,249.00,59000.00,Bajista
953,2024-05-07,7.12,7.29,6.98,64947.00,62312.08,25598.79,1272898.00,3005.69,298796.68,815246.00,576.50,289488.71,266127.00,6.98,7.13,7.52,6.98,6.44,45.10,21.00,25.00,31028.00,495,494,0.00,28.00,296.00,205.00,42000.00,Bajista
954,2024-05-08,6.99,7.19,6.89,75550.00,61193.03,26121.19,1415152.00,2974.21,266934.81,830635.00,588.60,297016.62,249379.00,6.99,7.12,7.52,6.99,6.46,44.94,17.00,24.00,32040.00,426,494,0.00,24.00,230.00,177.00,49000.00,Lateral
955,2024-05-09,6.98,7.09,6.78,75016.00,63074.01,30660.81,1381957.00,3036.23,238561.75,686147.00,596.80,464857.60,332988.00,7.01,7.11,7.52,7.01,6.50,46.32,18.00,17.00,29314.00,475,464,0.00,16.00,188.00,257.00,50000.00,Lateral


In [17]:
n_clases = 3 

validation_predictions = ensemble.predict(clasifier_validation.drop(columns=["Open_time", "Tendencia"]))
predicciones_one_hot = to_categorical(validation_predictions, num_classes=n_clases)
etiquetas_numericas = np.argmax(predicciones_one_hot, axis=1)
categorias_clases = onehot_encoder.categories_[0]
nombres_clases = [categorias_clases[indice] for indice in etiquetas_numericas]

display(validation_predictions)
display(categorias_clases)
display(nombres_clases)

1/1 [==============================] - 1s 1s/step


array([0, 1, 0, 0, 0, 0, 0, 1, 0, 0])

array(['Alcista', 'Bajista', 'Lateral'], dtype=object)

['Alcista',
 'Bajista',
 'Alcista',
 'Alcista',
 'Alcista',
 'Alcista',
 'Alcista',
 'Bajista',
 'Alcista',
 'Alcista']

##### Datos generados por auto ml con prophet:

In [18]:
auto_ml_prophet_df = pd.read_csv('auto_timeseries_models_prophet/predicciones.csv')
auto_mp_prophet_predictions = ensemble.predict(auto_ml_prophet_df.drop(columns=["Open_time", "Close"]))

predicciones_one_hot = to_categorical(auto_mp_prophet_predictions, num_classes=n_clases)
etiquetas_numericas = np.argmax(predicciones_one_hot, axis=1)
categorias_clases = onehot_encoder.categories_[0]
nombres_clases = [categorias_clases[indice] for indice in etiquetas_numericas]

display(auto_mp_prophet_predictions)
display(categorias_clases)
display(nombres_clases)

1/1 [==============================] - 0s 202ms/step


array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

array(['Alcista', 'Bajista', 'Lateral'], dtype=object)

['Alcista',
 'Alcista',
 'Bajista',
 'Bajista',
 'Bajista',
 'Bajista',
 'Bajista',
 'Bajista',
 'Bajista',
 'Alcista']

##### Datos generados por auto ml con stats:

In [20]:
auto_ml_stats_df = pd.read_csv('auto_timeseries_models/predicciones.csv')
auto_mp_stats_predictions = ensemble.predict(auto_ml_stats_df.drop(columns=["Open_time", "Close"]))

predicciones_one_hot = to_categorical(auto_mp_stats_predictions, num_classes=n_clases)
etiquetas_numericas = np.argmax(predicciones_one_hot, axis=1)
categorias_clases = onehot_encoder.categories_[0]
nombres_clases = [categorias_clases[indice] for indice in etiquetas_numericas]

display(auto_mp_stats_predictions)
display(categorias_clases)
display(nombres_clases)

1/1 [==============================] - 0s 190ms/step


array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0])

array(['Alcista', 'Bajista', 'Lateral'], dtype=object)

['Alcista',
 'Alcista',
 'Bajista',
 'Alcista',
 'Alcista',
 'Alcista',
 'Bajista',
 'Alcista',
 'Alcista',
 'Alcista']

##### Datos generados por auto ml con modelos clasicos:

In [19]:
auto_ml_df = pd.read_csv('h2o_models/predicciones.csv')
auto_mp_predictions = ensemble.predict(auto_ml_df.drop(columns=["Open_time", "Next_Day_Target", "Close"]))

predicciones_one_hot = to_categorical(auto_mp_predictions, num_classes=n_clases)
etiquetas_numericas = np.argmax(predicciones_one_hot, axis=1)
categorias_clases = onehot_encoder.categories_[0]
nombres_clases = [categorias_clases[indice] for indice in etiquetas_numericas]

display(auto_mp_predictions)
display(categorias_clases)
display(nombres_clases)

1/1 [==============================] - 0s 171ms/step


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

array(['Alcista', 'Bajista', 'Lateral'], dtype=object)

['Alcista',
 'Alcista',
 'Alcista',
 'Alcista',
 'Alcista',
 'Alcista',
 'Alcista',
 'Alcista',
 'Alcista',
 'Alcista']